# 01 Data Preparation
**German Energy Demand Forecasting**

## 📊 Data Sources:
- **SMARD / Bundesnetzagentur** Hourly realized electricity load for Germany, Time span: 2015–2024 [https://www.smard.de/home/downloadcenter/download-marktdaten/]
- **Weather data** Daily observations from selected DWD weather stations [https://www.dwd.de/DE/leistungen/klimadatendeutschland/klarchivstunden.html] 
- **Calender Data** Python `holidays` library

In [1]:
# Library Imports
import os
import numpy as np
import pandas as pd
import warnings
import holidays
from pathlib import Path

# Ignore Warnings
warnings.filterwarnings('ignore')

# Pfade setzen
os.chdir(Path('../data/raw').resolve())

## SMARD Energy Data (2015-2024)
Weather data is merged at the daily level using calendar dates. Missing values and time alignment issues (e.g. daylight saving time) are handled during this step to ensure consistency across time scales.

In [2]:
# Load Data

energy_1516 = pd.read_csv("Realisierter_Stromverbrauch_201501010000_201701010000_Stunde.csv", sep=";")
energy_1718 = pd.read_csv("Realisierter_Stromverbrauch_201701010000_201901010000_Stunde.csv", sep=";")
energy_1920 = pd.read_csv("Realisierter_Stromverbrauch_201901010000_202101010000_Stunde.csv", sep=";")
energy_2122 = pd.read_csv("Realisierter_Stromverbrauch_202101010000_202301010000_Stunde.csv", sep=";")
energy_2324 = pd.read_csv("Realisierter_Stromverbrauch_202301010000_202501010000_Stunde.csv", sep=";")

energy_hour = pd.concat([energy_1516, energy_1718, energy_1920, energy_2122, energy_2324], ignore_index=True)

# Set Index

energy_hour["Datetime"] = pd.to_datetime(energy_hour["Datum von"], dayfirst=True)
energy_hour = energy_hour.set_index("Datetime").sort_index()

# Daylight Saving Time Fix

print(f"Raw Shape: {energy_hour.shape}")
dupes = energy_hour[energy_hour.index.duplicated(keep=False)]
print(f"Duplicates: {len(dupes)} observations")
energy_hour = energy_hour[~energy_hour.index.duplicated(keep="first")]
print(f"Clean Shape: {energy_hour.shape}")

# Clean Data

energy_hour = energy_hour.rename(columns={"Netzlast [MWh] Berechnete Auflösungen": "load_mwh"})
energy_hour["load_mwh"] = (
    energy_hour["load_mwh"]
    .astype(str)
    .str.replace(".", "", regex=False)
    .str.replace(",", ".", regex=False)
    .astype(float)
)

# Feature Engineering
energy_hour = energy_hour[["load_mwh"]].assign(
    hour=energy_hour.index.hour,
    date=energy_hour.index.normalize(),
    weekday=energy_hour.index.day_name(),
    month=energy_hour.index.month,
    year=energy_hour.index.year,
)

energy_hour["weekday"] = pd.Categorical(
    energy_hour["weekday"],
    categories=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
    ordered=True
)

print(f"Time Span: {energy_hour.index.min()} → {energy_hour.index.max()}")
print(f"Load Range: {energy_hour['load_mwh'].min():,.0f} - {energy_hour['load_mwh'].max():,.0f} MWh")
print("\nPreview:")
print(energy_hour.head())


Raw Shape: (87672, 6)
Duplicates: 20 observations
Clean Shape: (87662, 6)
Time Span: 2015-01-01 00:00:00 → 2024-12-31 23:00:00
Load Range: 30,903 - 81,320 MWh

Preview:
                     load_mwh  hour       date   weekday  month  year
Datetime                                                             
2015-01-01 00:00:00  44600.25     0 2015-01-01  Thursday      1  2015
2015-01-01 01:00:00  43454.75     1 2015-01-01  Thursday      1  2015
2015-01-01 02:00:00  41963.25     2 2015-01-01  Thursday      1  2015
2015-01-01 03:00:00  40617.75     3 2015-01-01  Thursday      1  2015
2015-01-01 04:00:00  39936.75     4 2015-01-01  Thursday      1  2015


### Daily Energy Aggregation

**Daily sums for Weather Merge + Long-term Analysis**

In [3]:
energy_day = (
    energy_hour
    .groupby("date", as_index=True)
    .agg(load_mwh=("load_mwh", "sum"))
)

# Feature Engineering
energy_day = energy_day.assign(
    weekday=energy_day.index.day_name(),
    dayofweek=energy_day.index.dayofweek,  
    week=energy_day.index.isocalendar().week.astype(int),
    month=energy_day.index.month,
    year=energy_day.index.year, 
)

energy_day["weekday"] = pd.Categorical(
    energy_day["weekday"],
    categories=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
    ordered=True
)

print(f"Daily Shape: {energy_day.shape}")
print(f"Daily Span: {energy_day.index.min()} → {energy_day.index.max()}")
print(f"Daily Range: {energy_day['load_mwh'].min():,.0f} - {energy_day['load_mwh'].max():,.0f} MWh")
print("\nDaily Preview:")
print(energy_day.head())

Daily Shape: (3653, 6)
Daily Span: 2015-01-01 00:00:00 → 2024-12-31 00:00:00
Daily Range: 864,718 - 1,729,547 MWh

Daily Preview:
              load_mwh   weekday  dayofweek  week  month  year
date                                                          
2015-01-01  1096852.75  Thursday          3     1      1  2015
2015-01-02  1288914.75    Friday          4     1      1  2015
2015-01-03  1213309.50  Saturday          5     1      1  2015
2015-01-04  1177896.00    Sunday          6     1      1  2015
2015-01-05  1425927.50    Monday          0     2      1  2015


## DWD Weather Data (2015-2024)

**Daily TMK (Temperature)**

5 Stations → Daily Mean Temperature

In [4]:
# Load Data

weather1 = pd.read_csv("klarchiv_00433_daily_his/produkt_klima_tag_19480101_20241231_00433.txt", sep=";")
weather2 = pd.read_csv("klarchiv_01078_daily_his/produkt_klima_tag_19520101_20241231_01078.txt", sep=";")
weather3 = pd.read_csv("klarchiv_01262_daily_his/produkt_klima_tag_19920517_20241231_01262.txt", sep=";")
weather4 = pd.read_csv("klarchiv_01975_daily_his/produkt_klima_tag_19360101_20241231_01975.txt", sep=";")
weather5 = pd.read_csv("klarchiv_04931_daily_his/produkt_klima_tag_19530101_20241231_04931.txt", sep=";")

weather = pd.concat([weather1, weather2, weather3, weather4, weather5], ignore_index=True)
weather.columns = weather.columns.str.strip()
weather["date"] = pd.to_datetime(weather["MESS_DATUM"], format="%Y%m%d")

# 2015-2024 + TMK only
weather = weather[
    (weather["date"] >= "2015-01-01") & 
    (weather["date"] <= "2024-12-31")
][["date", "STATIONS_ID", "TMK"]].copy()

# Missing Values (-999 → NaN)
weather["TMK"] = weather["TMK"].replace(-999, np.nan)

# Daily Mean Temperature (5 Stations)
weather_mean = weather.groupby("date")["TMK"].mean().reset_index()
weather_mean["date"] = pd.to_datetime(weather_mean["date"])

print(f"Weather Shape: {weather_mean.shape}")
print(f"Weather Span: {weather_mean['date'].min()} → {weather_mean['date'].max()}")
print(f"TMK Range: {weather_mean['TMK'].min():.1f}°C - {weather_mean['TMK'].max():.1f}°C")
print("\nWeather Preview:")
print(weather_mean.head())


Weather Shape: (3653, 2)
Weather Span: 2015-01-01 00:00:00 → 2024-12-31 00:00:00
TMK Range: -9.0°C - 27.8°C

Weather Preview:
        date   TMK
0 2015-01-01  1.00
1 2015-01-02  3.28
2 2015-01-03  3.20
3 2015-01-04  2.76
4 2015-01-05  2.36


## Data-driven Holiday Detection

**IQR Outlier Analysis**

1. **National Holidays** (`holidays.DE()`): 36/38 weekday-outliers explained 
2. **Remaining 2 outliers** → Fronleichnam (2023-06-08, 2024-05-30)  
3. **Final `is_holiday`**: National + Fronleichnam → 38/38 coverage


In [5]:
# National Holidays
de_holidays = holidays.DE(years=range(2015, 2025))
energy_day['is_holiday'] = energy_day.index.map(
    lambda x: 1 if x in de_holidays else 0
)

# IQR Outlier Analysis (by weekday)
outliers = []
for wd, df_wd in energy_day.groupby('weekday'):
    q1 = df_wd['load_mwh'].quantile(0.25)
    q3 = df_wd['load_mwh'].quantile(0.75)
    iqr = q3 - q1
    lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    
    df_out = df_wd[(df_wd['load_mwh'] < lower) | (df_wd['load_mwh'] > upper)]
    outliers.append(df_out)

outliers = pd.concat(outliers)

# Check Outliers
outliers['is_holiday'] = outliers.index.normalize().map(
    lambda x: 1 if x in de_holidays else 0
)
print(f"{outliers['is_holiday'].sum():,} / {len(outliers)} = 94.7% → National Holidays\n")
print("Remaining Outliers (Fronleichnam):")
print(outliers[outliers['is_holiday']==0])

# REGIONAL: Fronleichnam
fronleichnam_dates = {key: val for key, val in holidays.DE(subdiv = "NW", years=range(2015, 2025)).items() if val == 'Fronleichnam'}
de_holidays = set(de_holidays.keys()) | set(fronleichnam_dates.keys())


36 / 38 = 94.7% → National Holidays

Remaining Outliers (Fronleichnam):
             load_mwh   weekday  dayofweek  week  month  year  is_holiday
date                                                                     
2023-06-08  1072759.0  Thursday          3    23      6  2023           0
2024-05-30  1097436.0  Thursday          3    22      5  2024           0


## Merge & Save

In [6]:
# Hourly Dataset
energy_weather_hour = energy_hour.merge(
    weather_mean.set_index('date'), 
    left_on='date', 
    right_index=True, 
    how="left"
)
energy_weather_hour['is_holiday'] = energy_weather_hour.index.map(
    lambda x: 1 if x.date() in de_holidays else 0
)

# Daily Dataset
energy_weather_day = energy_day.merge(
    weather_mean.set_index('date'), 
    left_index=True, 
    right_index=True, 
    how="left"
)
energy_weather_day['is_holiday'] = energy_weather_day.index.map(
    lambda x: 1 if x.date() in de_holidays else 0
)

print(f"Hourly Shape: {energy_weather_hour.shape}")
print(f"Daily Shape:  {energy_weather_day.shape}")

print("\nPreview (Hourly):")
print(energy_weather_hour.head())

print("\nPreview (Daily):")
print(energy_weather_day.head())

Hourly Shape: (87662, 8)
Daily Shape:  (3653, 8)

Preview (Hourly):
                     load_mwh  hour       date   weekday  month  year  TMK  \
Datetime                                                                     
2015-01-01 00:00:00  44600.25     0 2015-01-01  Thursday      1  2015  1.0   
2015-01-01 01:00:00  43454.75     1 2015-01-01  Thursday      1  2015  1.0   
2015-01-01 02:00:00  41963.25     2 2015-01-01  Thursday      1  2015  1.0   
2015-01-01 03:00:00  40617.75     3 2015-01-01  Thursday      1  2015  1.0   
2015-01-01 04:00:00  39936.75     4 2015-01-01  Thursday      1  2015  1.0   

                     is_holiday  
Datetime                         
2015-01-01 00:00:00           1  
2015-01-01 01:00:00           1  
2015-01-01 02:00:00           1  
2015-01-01 03:00:00           1  
2015-01-01 04:00:00           1  

Preview (Daily):
              load_mwh   weekday  dayofweek  week  month  year  is_holiday  \
date                                               

In [7]:
# Save
energy_weather_hour.to_csv("../processed/energy_weather_hourly.csv", index_label="datetime")
energy_weather_day.to_csv("../processed/energy_weather_daily.csv", index_label="date")